In [1]:
%pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 4.0 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np

# === Train set ===
df_LTrain_x = pd.read_csv('arkomadataset/LeftArmDataset/LTrain_x.csv')
df_LTrain_y = pd.read_csv('arkomadataset/LeftArmDataset/LTrain_y.csv')

# === Validation set ===
df_LVal_x = pd.read_csv('arkomadataset/LeftArmDataset/LVal_x.csv')
df_LVal_y = pd.read_csv('arkomadataset/LeftArmDataset/LVal_y.csv')

# === Test set ===
df_LTest_x = pd.read_csv('arkomadataset/LeftArmDataset/LTest_x.csv')
df_LTest_y = pd.read_csv('arkomadataset/LeftArmDataset/LTest_y.csv')

# Just to see how many rows/columns you have
print("Train X shape:", df_LTrain_x.shape)
print("Train Y shape:", df_LTrain_y.shape)
print("Validation X shape:", df_LVal_x.shape)
print("Validation Y shape:", df_LVal_y.shape)
print("Test X shape:", df_LTest_x.shape)
print("Test Y shape:", df_LTest_y.shape)

Train X shape: (6000, 6)
Train Y shape: (6000, 5)
Validation X shape: (2000, 6)
Validation Y shape: (2000, 5)
Test X shape: (2000, 6)
Test Y shape: (2000, 5)


In [3]:
X_train = df_LTrain_x.values
y_train = df_LTrain_y.values

X_val = df_LVal_x.values
y_val = df_LVal_y.values

X_test = df_LTest_x.values
y_test = df_LTest_y.values

In [4]:
from sklearn.preprocessing import StandardScaler

# Create scalers for inputs and outputs
scaler_X = StandardScaler()
scaler_y = StandardScaler()
print(scaler_y)
# Fit scalers only on the training data and transform training, validation, and test sets:
X_train_scaled = scaler_X.fit_transform(X_train)
X_val_scaled = scaler_X.transform(X_val)
X_test_scaled = scaler_X.transform(X_test)


y_train_scaled = scaler_y.fit_transform(y_train)
y_val_scaled = scaler_y.transform(y_val)
y_test_scaled = scaler_y.transform(y_test)

print("Sample normalized input:", X_train_scaled[0])
print("Sample normalized output:", y_train_scaled[0])

StandardScaler()
Sample normalized input: [-1.66424774  0.04971416 -1.27395635 -0.0678829   1.86054145  0.13048967]
Sample normalized output: [ 1.23094922  0.60199925  0.84784477 -0.60489306 -0.75798009]


In [5]:
import tensorflow as tf

# New differentiable forward kinematics for a 5-DOF arm:
def forward_kinematics_tf_5(joint_angles, link_lengths):
    """
    Computes the end-effector position for a 5-DOF robotic arm.

    Parameters:
      joint_angles: Tensor of shape [batch, 5]
      link_lengths: list or array of 5 link lengths, e.g., [l1, l2, l3, l4, l5]

    Returns:
      A tensor of shape [batch, 3] representing the (x, y, z) end-effector position.
    """
    # Unpack joint angles for clarity
    theta1 = joint_angles[:, 0]  # rotation in XY plane
    theta2 = joint_angles[:, 1]  # vertical motion
    theta3 = joint_angles[:, 2]  # further extension
    theta4 = joint_angles[:, 3]
    theta5 = joint_angles[:, 4]

    # For simplicity, assume:
    # - First link rotates in XY.
    # - Each subsequent link extends the arm, adding cumulative rotations.
    # Base position:
    x0 = tf.zeros_like(theta1)
    y0 = tf.zeros_like(theta1)
    z0 = tf.zeros_like(theta1)

    # First link:
    x1 = x0 + link_lengths[0] * tf.cos(theta1)
    y1 = y0 + link_lengths[0] * tf.sin(theta1)
    z1 = z0  # no vertical movement yet

    # Second link:
    x2 = x1 + link_lengths[1] * tf.cos(theta1) * tf.cos(theta2)
    y2 = y1 + link_lengths[1] * tf.sin(theta1) * tf.cos(theta2)
    z2 = z1 + link_lengths[1] * tf.sin(theta2)

    # Third link:
    x3 = x2 + link_lengths[2] * tf.cos(theta1) * tf.cos(theta2 + theta3)
    y3 = y2 + link_lengths[2] * tf.sin(theta1) * tf.cos(theta2 + theta3)
    z3 = z2 + link_lengths[2] * tf.sin(theta2 + theta3)

    # Fourth link:
    x4 = x3 + link_lengths[3] * tf.cos(theta1) * tf.cos(theta2 + theta3 + theta4)
    y4 = y3 + link_lengths[3] * tf.sin(theta1) * tf.cos(theta2 + theta3 + theta4)
    z4 = z3 + link_lengths[3] * tf.sin(theta2 + theta3 + theta4)

    # Fifth link:
    x5 = x4 + link_lengths[4] * tf.cos(theta1) * tf.cos(theta2 + theta3 + theta4 + theta5)
    y5 = y4 + link_lengths[4] * tf.sin(theta1) * tf.cos(theta2 + theta3 + theta4 + theta5)
    z5 = z4 + link_lengths[4] * tf.sin(theta2 + theta3 + theta4 + theta5)

    return tf.stack([x5, y5, z5], axis=1)  # shape: [batch, 3]



In [14]:
# Custom PINN model for 5-DOF outputs:
class PINNModel(tf.keras.Model):
    def __init__(self, link_lengths, lambda_phys=0.1, **kwargs):
        super(PINNModel, self).__init__(**kwargs)
        self.link_lengths = link_lengths  # Expecting a list of 5 link lengths
        self.lambda_phys = lambda_phys    # Weight for physics loss
        # Define network layers
        self.d1 = tf.keras.layers.Dense(64, activation='relu')
        self.d2 = tf.keras.layers.Dense(64, activation='relu')
        self.dropout = tf.keras.layers.Dropout(0.2)
        self.d3 = tf.keras.layers.Dense(32, activation='relu')
        # Updated: output 5 values (instead of 3)
        self.out_layer = tf.keras.layers.Dense(5, activation='linear')

    def call(self, inputs, training=False):
        x = self.d1(inputs)
        x = self.d2(x)
        if training:
            x = self.dropout(x, training=training)
        x = self.d3(x)
        return self.out_layer(x)

    def train_step(self, data):
        # Unpack data: x = input features, y = ground truth joint angles (shape [batch, 5])
        x, y = data
        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)  # predicted joint angles, shape [batch, 5]

            # Standard data loss (MSE) between predictions and true joint angles
            loss_data = tf.reduce_mean(tf.square(y - y_pred))

            # Physics loss:
            # - Compute predicted end-effector position using our differentiable forward kinematics.
            # - Assume x[:, :3] contains the target [x,y,z] of the end-effector.
            pred_positions = forward_kinematics_tf_5(y_pred, self.link_lengths)
            target_positions = x[:, :3]
            loss_phys = tf.reduce_mean(tf.square(target_positions - pred_positions))

            # Total loss: data loss plus weighted physics loss
            loss = loss_data + self.lambda_phys * loss_phys

        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        return {"loss": loss, "data_loss": loss_data, "phys_loss": loss_phys}



In [15]:
# Example usage in Google Colab:

# Define link lengths for a 5-DOF arm (you may adjust these values)
link_lengths = [1.0, 1.0, 0.8, 0.5, 0.3]
pinn_model = PINNModel(link_lengths=link_lengths, lambda_phys=0.1)

# Compile the model
pinn_model.compile(optimizer='adam', loss=lambda y_true, y_pred: tf.constant(0.0))


# Now train the model using your prepared data.
# Make sure X_train_scaled has target positions in its first 3 columns.
# Also ensure that y_train_scaled has 5 columns (the first 5 columns of your original y targets).
history_pinn = pinn_model.fit(X_train_scaled, y_train_scaled,
                              epochs=200, batch_size=64,
                              validation_data=(X_val_scaled, y_val_scaled))

# # Save the model to download
# pinn_model.save("pinn_model.h5")


Epoch 1/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 8s 33ms/step - data_loss: 0.6902 - loss: 0.9856 - phys_loss: 2.9542 - val_loss: 0.0000e+00
Epoch 2/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - data_loss: 0.5085 - loss: 0.7337 - phys_loss: 2.2515 - val_loss: 0.0000e+00
Epoch 3/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - data_loss: 0.4109 - loss: 0.6257 - phys_loss: 2.1478 - val_loss: 0.0000e+00
Epoch 4/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - data_loss: 0.3326 - loss: 0.5435 - phys_loss: 2.1095 - val_loss: 0.0000e+00
Epoch 5/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - data_loss: 0.2865 - loss: 0.4923 - phys_loss: 2.0579 - val_loss: 0.0000e+00
Epoch 6/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - data_loss: 0.2586 - loss: 0.4632 - phys_loss: 2.0457 - val_loss: 0.0000e+00
Epoch 7/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - data_loss: 0.2456 - loss: 0.4486 - phys_loss: 2.0299 - val_loss: 0.0000e+00
Epoch 8/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - data_loss: 0.2330 - loss: 0.4346 - phys_los

In [16]:
# Cell 4: import Keras Tuner & define build_pinn
import keras_tuner as kt
from tensorflow import keras

def build_pinn(hp):
    # sample hyperparameters
    units1 = hp.Int("units1", 32, 128, step=32)
    units2 = hp.Int("units2", 32, 128, step=32)
    lr     = hp.Float("lr", 1e-4, 1e-2, sampling="log")
    drop   = hp.Float("drop", 0.0, 0.5, step=0.1)
    lam    = hp.Float("lambda_phys", 0.0, 1.0, step=0.1)

    # build your PINNModel
    model = PINNModel(link_lengths, lambda_phys=lam)
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=lr),
        loss=lambda y_true, y_pred: tf.constant(0.0)
    )
    return model


In [18]:
# Cell 5: set up Bayesian tuner and search
tuner = kt.BayesianOptimization(
    build_pinn,
    objective=kt.Objective("loss", direction="min"),
    max_trials=20,
    num_initial_points=5,
    directory="kt_dir",
    project_name="pinn_bayes"
)

tuner.search(
    X_train_scaled, y_train_scaled,
    epochs=100,
    validation_data=(X_val_scaled, y_val_scaled),
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)



Trial 20 Complete [00h 00m 11s]
loss: 0.0899258479475975

Best loss So Far: 0.06188662722706795
Total elapsed time: 00h 04m 45s


In [29]:
# Cell 6: retrieve best hyperparameters and rebuild
best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = build_pinn(best_hp)
best_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=best_hp.get("lr")),
    loss="mse",
    metrics=["mae"]
)

# … you can now retrain best_model or evaluate directly

# Cell 6: Retrieve and save the best model
# (place after tuner.search in your notebook)

# 1. Get the single best Keras model
# best_model = tuner.get_best_models(num_models=1)[0]

# Cell 6: Evaluate on your held-out test set
# If you trained the best model via build_pinn(best_hp), call that:
test_loss, test_mae = best_model.evaluate(X_test_scaled, y_test_scaled)

print("Test MSE:", test_loss)
print("Test MAE:", test_mae)


# 3. Save to disk in HDF5 or SavedModel format
best_model.save('pinn_model.keras')
# best_model.save("pinn_model.h5")              # HDF5 file :contentReference[oaicite:6]{index=6}
# or
# best_model.save("pinn_best_model_savedmodel",        # SavedModel directory
#                 save_format="tf")



63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 1.3204 - mae: 0.9148
Test MSE: 1.1641297340393066
Test MAE: 0.8622229695320129
